In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ['TRANSFORMERS_CACHE'] = '/mnt/iMVR/junde/.cache/huggingface/hub'

from langchain.llms import HuggingFacePipeline
import networkx as nx
from transformers import pipeline
import re
from creat import creat_world
from prompt import *

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pipeline = pipeline(model="meta-llama/Llama-2-7b-chat-hf", device_map="auto")
model = HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0.7})

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.08s/it]


In [3]:
locations, agents = creat_world(model)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, sup

In [4]:
global_time = 0
for repeats in range(5):
  global_time += 1
  print("In global time", global_time)
  action_prompts = {}
  action_results = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)
    
    # for name in people:
      # prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
    people_description = []
    for i in people: 
      if i not in action_results: # initialize action results as the observations
        action_results[i] = agents[i].get_summary(force_refresh=False)
      people_description.append(i+': '+ action_results[i])


    for i in people: # add observation to memory and react
      print("Mind Tree of people: ", i)

      others = [x for x in people if x != i]
      observation = "You are {}.You are currently in {} with the following description: {}. \
      It is currently {}:00. The following people are in this area: {}. You can interact with them.". \
      format(i, location, town_areas[location], str(global_time), ', '.join(others))

      others_des = [x for x in people_description if i+': ' not in x]
      observation += ' You know the following about people: ' + '. '.join(others_des)

      # print("for people", i)
      # print("the observation is:", observation)

      agents[i].memory.add_memory(observation)
      _, reaction = agents[i].generate_reaction(observation)
      action_results[i] = reaction

      print("action result is:")
      print(reaction)

In global time 1
Mind Tree of people:  Toblen Stonehill


agent_summary_description is: Name: Toblen Stonehill (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Toblen owns a trading post.
end
relevant_mem is: You are in Phandalin Town Square. The town is small and has a population of around 500 people. The town is surrounded by a dense forest and is located near a river. The town is known for its apple orchards and is a popular stop for travelers. There is a trading post in the town square where you can buy supplies. There is also a tavern where you can rest and gather information. The town is currently at peace, but there have been rumors of bandits and monsters in the nearby forest.
# Goals:
         Your goal is to gather information about the nearby forest and the rumors of bandits and monsters.
# Objectives:
         Objective 1: Talk to Daran Edermath and learn more about the rumors of bandits and monsters in the nearby forest.
         Objective 2: Visit the trading post and purchase supplies for your journey.
         Objecti

Token indices sequence length is longer than the specified maximum sequence length for this model (1032 > 1024). Running this sequence through the model will result in indexing errors


full action logic
What would you like to do?
end
action result is:
 
Mind Tree of people:  Daran Edermath
agent_summary_description is: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.
end
relevant_mem is: * You are in Phandalin Town Square.
         * You are Daran Edermath.
         * It is December 08, 2023.
         * The time is 1:00 PM.
         * The weather is clear.
         * You have 15 gold pieces.
         * You have a backpack containing a bedroll, 10 days of rations, and a waterskin.
         * You have a belt pouch containing a small amount of healing potion.
         * You have a dagger for s

/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...

agent_summary_description is: Name: Terrill Bloodscar (age: 16)
Innate traits: ignorant
Terrill is a human ruffian. He wears a grimy scarlet cloak. He is a member of the Redbrand ruffians. He doesn't like adventurers, and wants to rob and kill them.
end
relevant_mem is: 1. You are Terrill Bloodscar.
        2. You are in Phandalin Town Square.
        3. It is currently 4:00 PM.
        4. The following people are in this area: Toblen Stonehill, Daran Edermath, Linene Graywind, Harbin Wester.
        5. You know the following about people: Toblen Stonehill: Name: Toblen Stonehill (age: 69)
Innate traits: mean, gossipy, arrogant, selfish
Toblen owns a trading post.. Daran Edermath: Name: Daran Edermath (age: 55)
Innate traits: Stubborn, upright, inarticulate, calm
Daran is a retired adventurer who lives in a tidy little cottage beside an apple
end


/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


full action logic
Terrill Bloodscar's status: chating with Redbrand ruffians members, secretly planing the robbery.

Terrill Bloodscar's reaction:
REACT: Terrill Bloodscar's eyes narrow as he listens to the townsfolk's conversation, his mind racing with potential schemes.
end


/mnt/iMVR/junde/miniconda3/lib/python3.11/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


action result is:
Terrill Bloodscar Terrill Bloodscar's eyes narrow as he listens to the townsfolk's conversation, his mind racing with potential schemes.
